In [1]:
# import pandas, numpy, scaler, kfold, linregression
import pandas as pd
import numpy as np
import zipfile
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [2]:
'''
1. Download the TF-DNA binding gcPBM data for Max, Myc, and Mad from 
Blackboard: ‘Max.txt’ for TF Max, ‘Mad.txt’ for TF Mad, and ‘Myc.txt’ 
for TF Myc. Each file contains two columns, where the first column 
represents the binding sequence and the second column shows the 
binding affinity. Utilize pandas to read these files into a DataFrame 
format. Once the data is loaded, count and report the number of 
sequences for each TF. [1pt]
'''

# import files
# import mad
file_path = '/Users/peytonhall/downloads/Mad.txt'
madDf = pd.read_csv(file_path, sep='\t', header=None)
madDf.columns = ['sequence', 'affinity']
madDf
# mad has 7534 sequences

# import max
file_path = '/Users/peytonhall/downloads/Max.txt'
maxDf = pd.read_csv(file_path, sep='\t', header=None)
maxDf.columns = ['sequence', 'affinity']
maxDf
# max has 8568 sequences

# import myc
file_path = '/Users/peytonhall/downloads/Myc.txt'
mycDf = pd.read_csv(file_path, sep='\t', header=None)
mycDf.columns = ['sequence', 'affinity']
mycDf
# myc has 6926 sequences

,sequence,affinity
0,ACCGACCGGCGCGGGCACGAGGCAATGGCGGCCGGG,7.847372
1,AACAGCGCCACCGGCCTCGTGCACTTCTTCCACTGT,8.027477
2,GCGGCCGGTCTGCACCATGCTGCGAACGTCCGTCCT,7.675546
3,TTATTGAGCCCCTACCATGTGCCAGGCCCTGGGCTA,8.419580
4,CCCCTCAGCTGCTTCCTCGTGGCGCTGATCATCTGG,8.453188
...,...,...
6921,GCCCCCTGCGGGAGCCGCGTGGACCGCCCCGGTGGC,8.746080
6922,GGACGTCTCAAAACACACGTGTGACTTAGATGGTAA,9.292289
6923,TCTGCCCAGTCCCTCCACATGGGAAATATAGCACTG,8.349484
6924,AGGACTACAAGTTCCAGCGTGCTTTGCTCGATTTCG,7.362645


In [3]:
'''
Write a function to convert DNA sequences into a one-hot 
encoding format. This should be done for both 
mononucleotide (1mer) and dinucleotide (2mer) formats.
'''

# one hot encoding for mad
# make dictionary to encode mononucleotide
def encodeMono(sequence):
    monoMap = {"A": [1, 0, 0, 0], "C": [0, 1, 0, 0], "G": [0, 0, 1, 0], "T": [0, 0, 0, 1]}
    encoded = []
    for mononucleotide in sequence:
        encoded.extend(monoMap[mononucleotide])
    return encoded

# encode dimer
def encodeDi(sequence):
    diMap = {"AA": [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], "AC": [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0], "AG": [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0], "AT": [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0], 
             "CA": [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0], "CC": [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0], "CG": [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0], "CT": [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0], 
             "GA": [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0], "GC": [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0], "GG": [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0], "GT": [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
             "TA": [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0], "TC": [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0], "TG": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0], "TT": [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]}
    encodedDi = []
    for dinucleotide in range(len(sequence)):
        if dinucleotide == len(sequence)-1:
            break
        diCount = sequence[dinucleotide: dinucleotide + 2] # to account for end of one dinucleotide starting another
        encodedDi.extend(diMap[diCount])
    return encodedDi

In [4]:
# convert sequence and affinity to lists
madSequences = madDf['sequence'].tolist()
madAffinity = madDf['affinity'].tolist()

maxSequences = maxDf['sequence'].tolist()
maxAffinity = maxDf['affinity'].tolist()

mycSequences = mycDf['sequence'].tolist()
mycAffinity = mycDf['affinity'].tolist()

In [5]:
# one hot encode the sequences
# one hot encode for mad
madMono = np.array([encodeMono(seq) for seq in madSequences])

# X = np.vstack((madMono, mgw))

madDi = np.array([encodeDi(seq) for seq in madSequences])
# one hot encode for max
maxMono = np.array([encodeMono(seq) for seq in maxSequences])
maxDi = np.array([encodeDi(seq) for seq in maxSequences])
# one hot encode for myc
mycMono = np.array([encodeMono(seq) for seq in mycSequences])
mycDi = np.array([encodeDi(seq) for seq in mycSequences])

In [7]:
print(madDi)

[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [68]:
# normalize binding affinity values using min-max normalization
# made a function to be able to apply to mad,max,myc
def normalizationFunct(affinityValues):
    minAffinityNorm = min(affinityValues)
    maxAffinityNorm = max(affinityValues)
    y = [(x - minAffinityNorm) / (maxAffinityNorm - minAffinityNorm) for x in affinityValues]
    y = np.array(y)
    return y

In [69]:
# apply normalization function to max,mad,myc
madAffinity = normalizationFunct(madAffinity)
maxAffinity = normalizationFunct(maxAffinity)
mycAffinity = normalizationFunct(mycAffinity)

In [70]:
madAffinity

array([0.41340591, 0.18571892, 0.79608383, ..., 0.16059828, 0.43706897,
       0.3819514 ])

In [72]:
'''Use 10-fold cross validation to train these two models. [1pt]'''
def training(X, y):
    # trainMadMono = trainingFunct(madMono, madAffinity)
    # trainMadDi = trainingFunct(madDi, madAffinity)
    # Configure the cross-validation
    cv = KFold(n_splits=10, shuffle=True, random_state=1)

    # results
    mse_scores = []; r_squared_scores = []

    # Initialize Linear Regression model
    model = LinearRegression()

    # Loop over each fold
    for train_index, valid_index in cv.split(X):
        # Split data
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]

        # Fit the model
        model.fit(X_train, y_train)

        # Predict on valid set
        predictions = model.predict(X_valid)

        # Evaluate the model
        mse = mean_squared_error(y_valid, predictions)
        mse_scores.append(mse)

        r_squared = model.score(X_valid, y_valid)
        r_squared_scores.append(r_squared)

    average_mse = np.mean(mse_scores)
    average_r_squared = np.mean(r_squared_scores)

    print(f'Average MSE: {average_mse} Average r_squared: {average_r_squared}')
    return 
    # each should give average MSE and r^2

In [73]:
# myc training

trainMycMono = training(mycMono, mycAffinity)
print(trainMycMono)


trainMycDi = training(mycDi, mycAffinity)
print(trainMycDi)

'''
Average MSE: 0.008063636558478526 Average r_squared: 0.7758656780550961
None
Average MSE: 0.006069934061210675 Average r_squared: 0.8312216101859413
None
'''

Average MSE: 0.008063636558478526 Average r_squared: 0.7758656780550961
None
Average MSE: 0.006069934061210675 Average r_squared: 0.8312216101859413
None


In [74]:
# mad training
trainMadMono = training(madMono, madAffinity)
print(trainMadMono)

trainMadDi = training(madDi, madAffinity)
print(trainMadDi)

'''
Average MSE: 0.010164454225023717 Average r_squared: 0.7731972103927929
None
Average MSE: 0.006738805918059245 Average r_squared: 0.8495551112457168
None
'''

Average MSE: 0.010164454225023717 Average r_squared: 0.7731972103927929
None
Average MSE: 0.006738805918059245 Average r_squared: 0.8495551112457168
None


In [75]:
# max training
trainMaxMono = training(maxMono, maxAffinity)
print(trainMaxMono)

trainMaxDi = training(maxDi, maxAffinity)
print(trainMaxDi)

'''
Average MSE: 0.006241655236634816 Average r_squared: 0.7833283773670303
None
Average MSE: 0.0040006941428690395 Average r_squared: 0.8610443815364244
None
'''

Average MSE: 0.006241655236634816 Average r_squared: 0.7833283773670303
None
Average MSE: 0.0040006941428690395 Average r_squared: 0.8610443815364244
None


In [ ]:
'''

'''